In [1]:
import pandas as pd
import requests
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, GlobalMaxPool1D, Dropout
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras import optimizers
import pickle
from data_cleaner import overall_cleaner

Using TensorFlow backend.


In [32]:
data_review = [json.loads(line) for line in open('datasets/review.json', 'r')]
data_business = [json.loads(line) for line in open('datasets/business.json', 'r')]
data_user = [json.loads(line) for line in open('datasets/user.json', 'r')]

review_df = pd.DataFrame(data_review)
biz_df = pd.DataFrame(data_business)
user_df = pd.DataFrame(data_user)

review_df.rename(columns={'stars': 'review_rating', 'useful':'review_usefulness'}, inplace=True)
user_df.rename(columns={'fans':'user_fans', 'funny':'user_funny', 'review_count': 'user_review_count', 'useful':'user_useful'}, inplace=True)
biz_df.rename(columns={'review_count':'review_count_biz', 'stars':'biz_stars'}, inplace=True)

r_u_df = review_df.merge(user_df,on = "user_id")
r_u_b_df = r_u_df.merge(biz_df, on = "business_id")
r_u_b_df.columns

Index(['business_id', 'cool_x', 'date', 'funny', 'review_id', 'review_rating',
       'text', 'review_usefulness', 'user_id', 'average_stars',
       'compliment_cool', 'compliment_cute', 'compliment_funny',
       'compliment_hot', 'compliment_list', 'compliment_more',
       'compliment_note', 'compliment_photos', 'compliment_plain',
       'compliment_profile', 'compliment_writer', 'cool_y', 'elite',
       'user_fans', 'friends', 'user_funny', 'name_x', 'user_review_count',
       'user_useful', 'yelping_since', 'address', 'attributes', 'categories',
       'city', 'hours', 'is_open', 'latitude', 'longitude', 'name_y',
       'postal_code', 'review_count_biz', 'biz_stars', 'state'],
      dtype='object')

In [35]:
r_u_b_df.head()

,business_id,cool_x,date,funny,review_id,review_rating,text,review_usefulness,user_id,average_stars,...,city,hours,is_open,latitude,longitude,name_y,postal_code,review_count_biz,biz_stars,state
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,2.00,...,Las Vegas,"{'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', '...",1,36.215546,-115.248798,MountainView Hospital,89128,238,2.5,NV
1,ujmEBvifdJM6h6RLv4wQIg,0,2013-03-27 14:17:13,0,oy8f3bxyl7zZJFDQ5edtIA,1.0,6 plus hours in the ER on a Tuesday Morning gr...,3,hG7b0MtEbXx5QzbzE6C_VA,2.00,...,Las Vegas,"{'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', '...",1,36.215546,-115.248798,MountainView Hospital,89128,238,2.5,NV
2,ujmEBvifdJM6h6RLv4wQIg,1,2012-08-17 21:58:24,0,8F9500ycq3mvpjf0glbFFg,3.0,Nursing staff is ok. I'm still a bit leary ab...,8,tH0uKD-vNwMoEc3Xk3Cbdg,3.87,...,Las Vegas,"{'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', '...",1,36.215546,-115.248798,MountainView Hospital,89128,238,2.5,NV
3,ujmEBvifdJM6h6RLv4wQIg,2,2015-07-18 11:40:36,2,qH-fr3sCKO0WoryJy44SGQ,5.0,"A week ago, I had a robotic surgery at Mountai...",13,renPzRDqMZpMaHiCD_e1_A,3.82,...,Las Vegas,"{'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', '...",1,36.215546,-115.248798,MountainView Hospital,89128,238,2.5,NV
4,ujmEBvifdJM6h6RLv4wQIg,3,2015-04-06 14:45:57,0,SnDqhw1IsB34BW1J1Er4mw,4.0,"I've been to a few hospitals over the years, a...",2,renPzRDqMZpMaHiCD_e1_A,3.82,...,Las Vegas,"{'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', '...",1,36.215546,-115.248798,MountainView Hospital,89128,238,2.5,NV


In [37]:
review_text = overall_cleaner(r_u_b_df, ['text', 'state', 
                                         'review_rating', 'review_usefulness', 
                                         'user_fans', 'user_funny', 'user_review_count', 
                                         'user_useful', 'review_count_biz', 'biz_stars'])

states_l = []

for i in np.unique(r_u_b_df['state']):
    states_l.append(i)
    
nv_df = review_text.groupby('state').get_group('NV')
nv_df.drop('state', axis=1, inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
review_text.groupby('state').count().sort_values('text', ascending=False)

,text,review_rating,review_usefulness,user_fans,user_funny,user_review_count,user_useful,review_count_biz,biz_stars
state,,,,,,,,,
NV,2320491,2320491,2320491,2320491,2320491,2320491,2320491,2320491,2320491
AZ,2082951,2082951,2082951,2082951,2082951,2082951,2082951,2082951,2082951
ON,784461,784461,784461,784461,784461,784461,784461,784461,784461
NC,408060,408060,408060,408060,408060,408060,408060,408060,408060
OH,321345,321345,321345,321345,321345,321345,321345,321345,321345
PA,290097,290097,290097,290097,290097,290097,290097,290097,290097
QC,179039,179039,179039,179039,179039,179039,179039,179039,179039
WI,133660,133660,133660,133660,133660,133660,133660,133660,133660
AB,99639,99639,99639,99639,99639,99639,99639,99639,99639


In [39]:
nv_df = review_text.groupby('state').get_group('NV')
az_df = review_text.groupby('state').get_group('AZ')
nc_df = review_text.groupby('state').get_group('NC')
on_df = review_text.groupby('state').get_group("ON")
oh_df = review_text.groupby('state').get_group('OH')
pa_df = review_text.groupby('state').get_group('PA')
qc_df = review_text.groupby("state").get_group('QC')
wi_df = review_text.groupby('state').get_group('WI')
ab_df = review_text.groupby('state').get_group('AB')

In [41]:
pickle.dump(nv_df.sample(n=95000), open('by_states/df_nv.p', 'wb'))
pickle.dump(az_df.sample(n=95000), open('by_states/df_az.p', 'wb'))
pickle.dump(nc_df.sample(n=95000), open('by_states/df_nc.p', 'wb'))
pickle.dump(on_df.sample(n=95000), open('by_states/df_on.p', 'wb'))
pickle.dump(oh_df.sample(n=95000), open('by_states/df_oh.p', 'wb'))
pickle.dump(pa_df.sample(n=95000), open('by_states/df_pa.p', 'wb'))
pickle.dump(qc_df.sample(n=95000), open('by_states/df_qc.p', 'wb'))
pickle.dump(wi_df.sample(n=95000), open('by_states/df_wi.p', 'wb'))
pickle.dump(ab_df.sample(n=95000), open('by_states/df_ab.p', 'wb'))